In [1]:
!git clone https://github.com/Weifeng-Chen/control-a-video.git
%cd control-a-video
# !pip3 install -r requirements.txt
!pip3 install diffusers==0.14.0
!pip3 install  transformers==4.27.3
!pip3 install accelerate==0.21.0
!pip3 install xformers==0.0.16
!pip3 install imageio==2.33
!pip3 install decord==0.6.0
!pip3 install opencv-python
!pip3 install einops==0.6.0
!pip3 install -q "huggingface_hub==0.14.1"



Cloning into 'control-a-video'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (16/16), done.
remote: Total 105 (delta 15), reused 15 (delta 15), pack-reused 89 (from 1)
Receiving objects: 100% (105/105), 3.06 MiB | 60.23 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/control-a-video
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.2
    Uninstalling diffusers-0.32.2:
      Successfully uninstalled diffusers-0.32.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninsta

In [2]:
!git clone https://github.com/Reouth/SelectiveControlNet.git


Cloning into 'SelectiveControlNet'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 68 (delta 24), reused 55 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 3.57 MiB | 8.17 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
# import sys
# sys.path.append(".")

from SelectiveControlNet.src.image_utils import extract_two_frames, frames_to_video
from SelectiveControlNet.src.masking import get_foreground_images


In [2]:
import os
import cv2
import numpy as np
from PIL import Image
from torchvision.transforms import ToTensor
from google.colab import files

In [9]:

uploaded = files.upload()
video_path = list(uploaded.keys())[0]

frames = extract_two_frames(video_path, [10, 20])  # 2 key frames

fg_masked = get_foreground_images(frames)

frames_to_video(frames, "two_frames_video.mp4", fps=2)


Saving 3402765-uhd_4096_2160_25fps.mp4 to 3402765-uhd_4096_2160_25fps (1).mp4


In [ ]:
!python3 inference.py \
  --prompt "mermaid underwater, blue clothes, corals and fishes background"\
  --input_video two_frames_video.mp4 \
  --control_mode canny \
  --num_sample_frames 2 \
  --each_sample_frame 2 \
  --sampling_rate 1 \
  --video_scale 1.0 \
  --init_noise_thres 0.3 \
  --inference_step 50 \
  --guidance_scale 7.5 \
  --height 512 \
  --width 512
